In [1]:
import csv
from neo4j import GraphDatabase

In [2]:
#CLASE PAR CREAR NODO CARROS
class CLASE_NODO(object): 
    def __init__(self):
        self._driver = GraphDatabase.driver("bolt:neo4j://localhost:7687", auth=("neo4j", "neo4jj"), encrypted=False)
    def close(self):
        self._driver.close()
    def CREAR_NODO(self, message,make,model,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,
                      engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,
                      engine_size,fuel_system,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._VALIDAR_NODO, message,make,model,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,
                      engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,
                      engine_size,fuel_system,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price)
    #METODO PARA CREAR LOS NODOS CARROS
    @staticmethod
    def _VALIDAR_NODO(tx, message,make,model,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,
                      engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,
                      engine_size,fuel_system,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price):
        result = tx.run("CREATE("+make+":Carros {make:'"+make+"',model:'"+model+"',fuel_type:'"+fuel_type+"',aspiration:'"+aspiration
                        +"',num_of_doors:'"+num_of_doors+"',body_style:'"+body_style+"',drive_wheels:'"+drive_wheels
                        +"',engine_location:'"+engine_location+"',wheel_base:'"+wheel_base+"',length:'"+length+"',width:'"+width
                        +"',height:'"+height+"',curb_weight:'"+curb_weight+"',engine_type:'"+engine_type+"',num_of_cylinders:'"+num_of_cylinders+
                        "',engine_size:'"+engine_size+"',fuel_system:'"+fuel_system+"',compression_ratio:'"+compression_ratio
                        +"',horsepower:'"+horsepower+"',peak_rpm:'"+peak_rpm+"',city_mpg:'"+city_mpg+"',highway_mpg:'"+highway_mpg+"',price:"+price+"}) "
                        "SET "+make+".message = $message "
                        "RETURN "+make+".message + ', from node ' + id("+make+")", message=message)
    

In [3]:
#SE INICIALIZA LA CLASE DE LOS METODOS DE NEO4J
grafo=CLASE_NODO()
with open('E:/Users/Gabriel Leonardo Ch/Downloads/Documents/9ciclo/SISTEMAS EXPERTOS/2interciclo/car_dataset.csv', newline='') as File:  
    reader = csv.reader(File)  
    for row in reader:
        if str(row[1])=='make':
            pass
        elif str(row[23])!='?':            
            grafo.CREAR_NODO("Se crea el nod0 "+row[0],str(row[1]).replace("-","_"),str(row[2]).replace("-","_"),str(row[3]).replace("-","_"),str(row[4]).replace("-","_"),str(row[5]).replace("-","_"),str(row[6]).replace("-","_"),str(row[7]).replace("-","_"),str(row[8]).replace("-","_"),row[9],str(row[10]).replace("-","_"),str(row[11]).replace("-","_"),str(row[12]).replace("-","_"),str(row[13]).replace("-","_"),str(row[14]).replace("-","_"),str(row[15]).replace("-","_"),str(row[16]).replace("-","_"),str(row[17]).replace("-","_"),str(row[18]).replace("-","_"),str(row[19]).replace("-","_"),str(row[20]).replace("-","_"),str(row[21]).replace("-","_"),str(row[22]).replace("-","_"),row[23])
        elif str(row[23]=='?'):
            v = str(row[23]).replace("?","0")
            grafo.CREAR_NODO("Se crea el nod0 "+row[0],str(row[1]).replace("-","_"),str(row[2]).replace("-","_"),str(row[3]).replace("-","_"),str(row[4]).replace("-","_"),str(row[5]).replace("-","_"),str(row[6]).replace("-","_"),str(row[7]).replace("-","_"),str(row[8]).replace("-","_"),row[9],str(row[10]).replace("-","_"),str(row[11]).replace("-","_"),str(row[12]).replace("-","_"),str(row[13]).replace("-","_"),str(row[14]).replace("-","_"),str(row[15]).replace("-","_"),str(row[16]).replace("-","_"),str(row[17]).replace("-","_"),str(row[18]).replace("-","_"),str(row[19]).replace("-","_"),str(row[20]).replace("-","_"),str(row[21]).replace("-","_"),str(row[22]).replace("-","_"),v)


ServiceUnavailable: Failed to establish connection to IPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión)

In [ ]:
# MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r

#SE APLICA EL METODO DE KNN
#SE CREA EL GRAF DE LOS NODOS Y EL ATRIBUTO PARA LA SIMILTUD
CALL gds.graph.create(
    'GrafoC',
    {
        Carros: {
            label: 'Carros',
            properties: 'price'
        }
    },
    '*'
);
#CUANTO SE REQUERIERE DE MEMORIA
CALL gds.beta.knn.write.estimate('GrafoC', {
  nodeWeightProperty: 'price',
  writeRelationshipType: 'SIMILAR',
  writeProperty: 'score',
  topK: 1
})
YIELD nodeCount, bytesMin, bytesMax, requiredMemory

#SE EJECUTA EL ALGORITMO
CALL gds.beta.knn.stream('GrafoC', {
    topK: 1,
    nodeWeightProperty: 'price',
    randomSeed: 42,
    concurrency: 1,
    sampleRate: 1.0,
    deltaThreshold: 0.0
})
YIELD node1, node2, similarity
RETURN gds.util.asNode(node1).make AS Carros1, gds.util.asNode(node2).make AS Carros2, similarity
ORDER BY similarity DESCENDING, Carros1, Carros2

#
CALL gds.beta.knn.write('GrafoC', {
    writeRelationshipType: 'SIMILAR',
    writeProperty: 'score',
    topK: 1,
    randomSeed: 42,
    nodeWeightProperty: 'price'
})
YIELD nodesCompared, relationshipsWritten